### S20 - RUTINA GENERACIÓN DE DATOS ADICIONALES en el FORECAST.

Parte de los forecast executión que están en estado 20 (Ya Ejecutado el Forecast), Genera en los archivos locales de detalle los gráficos a nivel línea,

Los deja en archivos locales algoritmo_Pronostico_Extendido y Actualiza esl estado de 20 a 30  Forecast FINALIZADO

### RUTINA GENERADORA DE GRÁFICOS

1) Leer archivo Solicitudes_Compra
2) Leer datos adicionales y id relacionados
3) Leer datos adicionales de la T710_Estadis_Reposición
4) Generar GRAFICOS
5) Actulizar Estado en connexa

In [ ]:
# Solo importa lo necesario desde el módulo de funciones
from funciones_forecast import (
    get_execution_execute_by_status,
    get_execution,
    get_execution_by_status,
    Close_Connection,
    Open_Conn_Postgres,
    update_execution
)

import pandas as pd # uso localmente la lectura de archivos.
import ace_tools_open as tools

from dotenv import dotenv_values
secrets = dotenv_values(".env")
folder = secrets["FOLDER_DATOS"]

In [ ]:
import funciones_forecast
print(funciones_forecast.__file__)

In [ ]:


def extender_datos_forecast(algoritmo, name, id_proveedor):
    # Recuperar Historial de Ventas
    df_ventas = pd.read_csv(f'{folder}/{name}_Ventas.csv')
    df_ventas['Codigo_Articulo']= df_ventas['Codigo_Articulo'].astype(int)
    df_ventas['Sucursal']= df_ventas['Sucursal'].astype(int)
    df_ventas['Fecha']= pd.to_datetime(df_ventas['Fecha'])

    # Recuperar Maestro de Artículos
    articulos = pd.read_csv(f'{folder}/{name}_Articulos.csv')

    # Recuperando Forecast Calculado
    df_forecast = pd.read_csv(f'{folder}/{algoritmo}_Solicitudes_Compra.csv')
    df_forecast.fillna(0)   # Por si se filtró algún missing value
    print(f"-> Datos Recuperados del CACHE: {id_proveedor}, Label: {name}")
    
        # AGREGAR DATOS COMPLEMENTARIOS para Subir a CONNEXA
    conn =Open_Conn_Postgres()
    query = """
    SELECT code, name, id FROM public.fnd_site
    ORDER BY code 
    """
    # Ejecutar la consulta SQL
    stores = pd.read_sql(query, conn)
    # Intentar convertir el campo 'code' a int, eliminando las filas que no se puedan convertir
    stores = stores[pd.to_numeric(stores['code'], errors='coerce').notna()].copy()
    stores['code'] = stores['code'].astype(int)

    # Leer Dataframe de los PRODUCTOS
    conn =Open_Conn_Postgres()
    query = """
    SELECT ext_code, description, id FROM public.fnd_product
    ORDER BY ext_code;
    """
    # Ejecutar la consulta SQL
    products = pd.read_sql(query, conn)
    # Intentar convertir el campo 'code' a int, eliminando las filas que no se puedan convertir
    products = products[pd.to_numeric(products['ext_code'], errors='coerce').notna()].copy()
    products['ext_code'] = products['ext_code'].astype(int)

    #Unir los dataframes por Codigo_Articulo = ext_code
    df_merged = df_forecast.merge(products, left_on='Codigo_Articulo', right_on='ext_code', how='left')
    df_merged.rename(columns={'id': 'product_id'}, inplace=True)
    df_merged.drop(columns=['ext_code','description'], inplace=True)

    df_merged = df_merged.merge(stores, left_on = 'Sucursal', right_on='code', how='left')
    df_merged.rename(columns={'id': 'site_id'}, inplace=True)
    df_merged.drop(columns=['code','name'], inplace=True)
    
    # SUBIR INFORMACIÓN DE ARTICULOS y ESTIDISTICA REPOSICIÓN
    # Seleccionar las columnas requeridas en un nuevo dataframe  FALTA ,I_COSTO_ESTADISTICO,I_PRECIO_VTA
    columnas_seleccionadas = [
        'C_PROVEEDOR_PRIMARIO', 'C_ARTICULO', 'C_SUCU_EMPR', 'I_PRECIO_VTA',
        'I_COSTO_ESTADISTICO', 'Q_FACTOR_VENTA_ESP', 'Q_FACTOR_VTA_SUCU',
        'F_ULTIMA_VTA', 'Q_VENTA_30_DIAS', 'Q_VENTA_15_DIAS', 'Q_VENTA_DOMINGO', 'Q_TRANSF_PEND',
        'Q_DIAS_CON_STOCK', 'Q_REPONER', 'Q_REPONER_INCLUIDO_SOBRE_STOCK', 'Q_VENTA_DIARIA_NORMAL',
        'Q_DIAS_STOCK', 'Q_DIAS_SOBRE_STOCK', 'Q_DIAS_ENTREGA_PROVEEDOR'
    ]

    # Filtrar el dataframe con las columnas seleccionadas
    df_nuevo = articulos[columnas_seleccionadas].copy()
    df_nuevo['C_SUCU_EMPR']= df_nuevo['C_SUCU_EMPR'].astype(int)
    
    # Realizar la fusión de los DataFrames utilizando 'Sucursal' y 'Codigo_Articulo' como claves
    df_merged = df_merged.merge(
        df_nuevo, 
        left_on=['Sucursal', 'Codigo_Articulo'], 
        right_on=['C_SUCU_EMPR', 'C_ARTICULO'], 
        how='left'
    )
    df_merged.drop(columns=['C_SUCU_EMPR','C_ARTICULO'], inplace=True)
    
    return df_merged

In [ ]:
# RUTINA PRINCIPAL RECORRE FORECAST EXCEC con STATUS 20 y pasa a 30

fes = get_execution_by_status(20)

# Filtrar registros con supply_forecast_execution_status_id = 20  #FORECAST OK
for index, row in fes[fes["supply_forecast_execution_status_id"] == 20].iterrows():
    algoritmo = row["name"]
    name = algoritmo.split('_ALGO')[0]
    execution_id = row["id"]
    id_proveedor = row["ext_supplier_code"]
    print("Algoritmo: " + algoritmo + "  - Name: " + name + " exce_id:" + str(execution_id) + " id: Proveedor "+id_proveedor)
    
    try:
        # Llamar a la función que genera los gráficos y datos extendidos
        df_extendido = extender_datos_forecast(algoritmo, name, id_proveedor)

        # Guardar el archivo CSV
        file_path = f"{folder}/{algoritmo}_Pronostico_Extendido.csv"
        df_extendido.to_csv(file_path, index=False)
        print(f"Archivo guardado: {file_path}")

        # Actualizar el status_id a 40 en el DataFrame original
        fes.at[index, "supply_forecast_execution_status_id"] = 30
        # ✅ Actualizar directamente en la base de datos el estado a 30
        update_execution(execution_id, supply_forecast_execution_status_id=30)
        print(f"Estado actualizado a 30 para {execution_id}")

    except Exception as e:
        print(f"Error procesando {name}: {e}")



### SANDBOX

In [ ]:
name ='62_ARCOR'
algoritmo = '62_ARCOR_ALGO_05'
id_proveedor = '62'

# Recuperar Maestro de Artículos
articulos = pd.read_csv(f'{folder}/{name}_Articulos.csv')

# Recuperando Forecast Calculado
df_forecast = pd.read_csv(f'{folder}/{algoritmo}_Solicitudes_Compra.csv')
df_forecast.fillna(0)   # Por si se filtró algún missing value
print(f"-> Datos Recuperados del CACHE: {id_proveedor}, Label: {name}")

In [ ]:
id_proveedor = '189'
name= '189_BODEGAS_LOPEZ'
forecast_execution_id ='1c15b1e1-a00a-4da7-bf39-257a830a4f8a'
forecast_execution_execute_id ='xxx'
supplier_id = '8e6e66f7-2482-451b-a656-5a1999343186'
algoritmo = '189_BODEGAS_LOPEZ_ALGO_04'


df_forecast_ext = pd.read_csv(f'{folder}/{algoritmo}_Pronostico_Extendido.csv')
df_forecast_ext['Codigo_Articulo']= df_forecast_ext['Codigo_Articulo'].astype(int)
df_forecast_ext['Sucursal']= df_forecast_ext['Sucursal'].astype(int)
df_forecast_ext.fillna(0)   # Por si se filtró algún missing value


# Mostrar la tabla con los gráficos en base64

tools.display_dataframe_to_user(name="Forecast Extendido", dataframe=df_forecast_ext)

In [ ]:
fes = get_execution_by_status(40)

tools.display_dataframe_to_user(name="Pendientes Ejecución", dataframe=fes)

In [ ]:
#tools.display_dataframe_to_user(name="Pendientes Ejecución", dataframe=df_merged)

In [ ]:
# RECALCULAR Datos Faltantes en Lote de Archivos.
for index, row in fes[fes["supply_forecast_execution_status_id"] == 40].iterrows():
    algoritmo = row["name"]
    name = algoritmo.split('_ALGO')[0]
    execution_id = row["id"]
    id_proveedor = row["ext_supplier_code"]
    print("Algoritmo: " + algoritmo + "  - Name: " + name + " exce_id:" + str(execution_id) + " id: Proveedor "+id_proveedor)
    
    try:
        df_ventas = pd.read_csv(f'{folder}/{name}_Ventas.csv')
        df_ventas['Codigo_Articulo']= df_ventas['Codigo_Articulo'].astype(int)
        df_ventas['Sucursal']= df_ventas['Sucursal'].astype(int)
        df_ventas['Fecha']= pd.to_datetime(df_ventas['Fecha'])

        # Recuperar Maestro de Artículos
        articulos = pd.read_csv(f'{folder}/{name}_Articulos.csv')

        # Recuperando Forecast Calculado
        # df_forecast = pd.read_csv(f'{folder}/{algoritmo}_Solicitudes_Compra.csv')
        # df_forecast.fillna(0)   # Por si se filtró algún missing value
        # print(f"-> Datos Recuperados del CACHE: {id_proveedor}, Label: {name}")

        # Tomo el Calculado para pegar al final los datos.
        df_forecast = pd.read_csv(f'{folder}/{algoritmo}_Pronostico_Extendido.csv')
        df_forecast['Codigo_Articulo']= df_forecast_ext['Codigo_Articulo'].astype(int)
        df_forecast['Sucursal']= df_forecast_ext['Sucursal'].astype(int)
        df_forecast.fillna(0)   # Por si se filtró algún missing value


            # AGREGAR DATOS COMPLEMENTARIOS para Subir a CONNEXA
        conn =Open_Conn_Postgres()
        query = """
        SELECT code, name, id FROM public.fnd_site
        ORDER BY code 
        """
        # Ejecutar la consulta SQL
        stores = pd.read_sql(query, conn)
        # Intentar convertir el campo 'code' a int, eliminando las filas que no se puedan convertir
        stores = stores[pd.to_numeric(stores['code'], errors='coerce').notna()].copy()
        stores['code'] = stores['code'].astype(int)

        # Leer Dataframe de los PRODUCTOS
        conn =Open_Conn_Postgres()
        query = """
        SELECT ext_code, description, id FROM public.fnd_product
        ORDER BY ext_code;
        """
        # Ejecutar la consulta SQL
        products = pd.read_sql(query, conn)
        # Intentar convertir el campo 'code' a int, eliminando las filas que no se puedan convertir
        products = products[pd.to_numeric(products['ext_code'], errors='coerce').notna()].copy()
        products['ext_code'] = products['ext_code'].astype(int)

        #Unir los dataframes por Codigo_Articulo = ext_code
        df_merged = df_forecast.merge(products, left_on='Codigo_Articulo', right_on='ext_code', how='left')
        df_merged.rename(columns={'id': 'product_id'}, inplace=True)
        df_merged.drop(columns=['ext_code','description'], inplace=True)

        df_merged = df_merged.merge(stores, left_on = 'Sucursal', right_on='code', how='left')
        df_merged.rename(columns={'id': 'site_id'}, inplace=True)
        df_merged.drop(columns=['code','name'], inplace=True)

        # SUBIR INFORMACIÓN DE ARTICULOS y ESTIDISTICA REPOSICIÓN
        # Seleccionar las columnas requeridas en un nuevo dataframe  FALTA ,I_COSTO_ESTADISTICO,I_PRECIO_VTA
        columnas_seleccionadas = [
            'C_PROVEEDOR_PRIMARIO', 'C_ARTICULO', 'C_SUCU_EMPR', 'I_PRECIO_VTA',
            'I_COSTO_ESTADISTICO', 'Q_FACTOR_VENTA_ESP', 'Q_FACTOR_VTA_SUCU',
            'F_ULTIMA_VTA', 'Q_VENTA_30_DIAS', 'Q_VENTA_15_DIAS', 'Q_VENTA_DOMINGO', 'Q_TRANSF_PEND',
            'Q_DIAS_CON_STOCK', 'Q_REPONER', 'Q_REPONER_INCLUIDO_SOBRE_STOCK', 'Q_VENTA_DIARIA_NORMAL',
            'Q_DIAS_STOCK', 'Q_DIAS_SOBRE_STOCK', 'Q_DIAS_ENTREGA_PROVEEDOR'
        ]

        # Filtrar el dataframe con las columnas seleccionadas
        df_nuevo = articulos[columnas_seleccionadas].copy()
        df_nuevo['C_SUCU_EMPR']= df_nuevo['C_SUCU_EMPR'].astype(int)

        # Realizar la fusión de los DataFrames utilizando 'Sucursal' y 'Codigo_Articulo' como claves
        df_merged = df_merged.merge(
            df_nuevo, 
            left_on=['Sucursal', 'Codigo_Articulo'], 
            right_on=['C_SUCU_EMPR', 'C_ARTICULO'], 
            how='left'
        )
        df_merged.drop(columns=['C_SUCU_EMPR','C_ARTICULO'], inplace=True)
        
        # Guardar el archivo CSV
        file_path = f"{folder}/{algoritmo}_Pronostico_Extendido.csv"
        df_merged.to_csv(file_path, index=False)
        print(f"Archivo guardado: {file_path}")

    except Exception as e:
        print(f"Error procesando {name}: {e}")

In [ ]:
# ARREGLAR FORECAST EXTENDIDO

df_ventas = pd.read_csv(f'{folder}/{name}_Ventas.csv')
df_ventas['Codigo_Articulo']= df_ventas['Codigo_Articulo'].astype(int)
df_ventas['Sucursal']= df_ventas['Sucursal'].astype(int)
df_ventas['Fecha']= pd.to_datetime(df_ventas['Fecha'])

# Recuperar Maestro de Artículos
articulos = pd.read_csv(f'{folder}/{name}_Articulos.csv')

# Recuperando Forecast Calculado
# df_forecast = pd.read_csv(f'{folder}/{algoritmo}_Solicitudes_Compra.csv')
# df_forecast.fillna(0)   # Por si se filtró algún missing value
# print(f"-> Datos Recuperados del CACHE: {id_proveedor}, Label: {name}")

# Tomo el Calculado para pegar al final los datos.
df_forecast = pd.read_csv(f'{folder}/{algoritmo}_Pronostico_Extendido.csv')
df_forecast['Codigo_Articulo']= df_forecast_ext['Codigo_Articulo'].astype(int)
df_forecast['Sucursal']= df_forecast_ext['Sucursal'].astype(int)
df_forecast.fillna(0)   # Por si se filtró algún missing value


    # AGREGAR DATOS COMPLEMENTARIOS para Subir a CONNEXA
conn =Open_Conn_Postgres()
query = """
SELECT code, name, id FROM public.fnd_site
ORDER BY code 
"""
# Ejecutar la consulta SQL
stores = pd.read_sql(query, conn)
# Intentar convertir el campo 'code' a int, eliminando las filas que no se puedan convertir
stores = stores[pd.to_numeric(stores['code'], errors='coerce').notna()].copy()
stores['code'] = stores['code'].astype(int)

# Leer Dataframe de los PRODUCTOS
conn =Open_Conn_Postgres()
query = """
SELECT ext_code, description, id FROM public.fnd_product
ORDER BY ext_code;
"""
# Ejecutar la consulta SQL
products = pd.read_sql(query, conn)
# Intentar convertir el campo 'code' a int, eliminando las filas que no se puedan convertir
products = products[pd.to_numeric(products['ext_code'], errors='coerce').notna()].copy()
products['ext_code'] = products['ext_code'].astype(int)

#Unir los dataframes por Codigo_Articulo = ext_code
df_merged = df_forecast.merge(products, left_on='Codigo_Articulo', right_on='ext_code', how='left')
df_merged.rename(columns={'id': 'product_id'}, inplace=True)
df_merged.drop(columns=['ext_code','description'], inplace=True)

df_merged = df_merged.merge(stores, left_on = 'Sucursal', right_on='code', how='left')
df_merged.rename(columns={'id': 'site_id'}, inplace=True)
df_merged.drop(columns=['code','name'], inplace=True)

# SUBIR INFORMACIÓN DE ARTICULOS y ESTIDISTICA REPOSICIÓN
# Seleccionar las columnas requeridas en un nuevo dataframe  FALTA ,I_COSTO_ESTADISTICO,I_PRECIO_VTA
columnas_seleccionadas = [
    'C_PROVEEDOR_PRIMARIO', 'C_ARTICULO', 'C_SUCU_EMPR', 'I_PRECIO_VTA',
    'I_COSTO_ESTADISTICO', 'Q_FACTOR_VENTA_ESP', 'Q_FACTOR_VTA_SUCU',
    'F_ULTIMA_VTA', 'Q_VENTA_30_DIAS', 'Q_VENTA_15_DIAS', 'Q_VENTA_DOMINGO', 'Q_TRANSF_PEND',
    'Q_DIAS_CON_STOCK', 'Q_REPONER', 'Q_REPONER_INCLUIDO_SOBRE_STOCK', 'Q_VENTA_DIARIA_NORMAL',
    'Q_DIAS_STOCK', 'Q_DIAS_SOBRE_STOCK', 'Q_DIAS_ENTREGA_PROVEEDOR'
]

# Filtrar el dataframe con las columnas seleccionadas
df_nuevo = articulos[columnas_seleccionadas].copy()
df_nuevo['C_SUCU_EMPR']= df_nuevo['C_SUCU_EMPR'].astype(int)

# Realizar la fusión de los DataFrames utilizando 'Sucursal' y 'Codigo_Articulo' como claves
df_merged = df_merged.merge(
    df_nuevo, 
    left_on=['Sucursal', 'Codigo_Articulo'], 
    right_on=['C_SUCU_EMPR', 'C_ARTICULO'], 
    how='left'
)
df_merged.drop(columns=['C_SUCU_EMPR','C_ARTICULO'], inplace=True)

In [ ]:
# Guardar el archivo CSV
file_path = f"{folder}/{algoritmo}_Pronostico_Extendido.csv"
df_merged.to_csv(file_path, index=False)
print(f"Archivo guardado: {file_path}")

In [ ]:
# Leer Dataframe de FORECAST EXECUTION 
        
fes = get_execution_by_status(20)

# Mostrar la tabla con los gráficos en base64

tools.display_dataframe_to_user(name="Forecast execution by STATUS", dataframe=fes)